In [1]:
import open3d as o3d
import numpy as np
import os
import glob
import pandas as pd
import pymeshlab as pml
import math
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def viz_mesh(obj_file_path, vis_option = "world_axes"):   
        
        if isinstance(obj_file_path, str):
            mesh = o3d.io.read_triangle_mesh(obj_file_path)
        else:
             mesh = obj_file_path
        mesh.compute_vertex_normals() 
        if vis_option == "smoothshade":
            o3d.visualization.draw_geometries([mesh], width=1280, height=720)
        elif vis_option == "wireframe_on_shaded":
            o3d.visualization.draw_geometries([mesh], width=1280, height=720, mesh_show_wireframe=True)
        elif vis_option == "wireframe":
            # We first need to obtain a lineset of the wireframe if we don't want to render the mesh itself
            wireframe = o3d.geometry.LineSet.create_from_triangle_mesh(mesh) 
            o3d.visualization.draw_geometries([wireframe], width=1280, height=720)
        elif vis_option == "world_axes":
            # Display the mesh including a world axis system.

            # Create the endpoints of each line. Each line is unit-length.
            # For the world axes, the origin is shared by all lines. So we have 4 endpoints in total
            line_endpoints = [
                [0, 0, 0],
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1]
            ]

            # List of indices into the 'line_endpoints' list, which describes which endpoints form which line
            line_indices = [[0, 1], [0, 2], [0, 3]]

            # Create a line set from the endpoints and indices
            world_axes = o3d.geometry.LineSet(
                points=o3d.utility.Vector3dVector(line_endpoints),
                lines=o3d.utility.Vector2iVector(line_indices),
            )

            # Render the line set and the mesh
            o3d.visualization.draw_geometries([mesh, world_axes], width=1280, height=720)
        elif vis_option == "black_background":
            # Create visualizer
            vis = o3d.visualization.Visualizer()
            vis.create_window()
            vis.add_geometry(mesh)

            # Set render options (e.g. background color)
            opt = vis.get_render_option()
            opt.background_color = np.asarray([0, 0, 0])

            # Run the visualizer
            vis.run()
            vis.destroy_window()


In [5]:
viz_mesh("./resampledO3D/House\D00001.obj")

In [6]:
def flatten_mesh(mesh):
    mesh.vertices = o3d.utility.Vector3dVector(np.asarray(mesh.vertices) * np.array([0, 1., 1.]) )
    return mesh

In [8]:
#flatten and visualize mesh
mesh = o3d.io.read_triangle_mesh("./resampledO3D/House\D00001.obj")
flatten_mesh(mesh)
viz_mesh(mesh)

In [17]:
# check surface area of mesh
mesh.get_surface_area()

1.7266193447261242

In [10]:

mesh.compute_vertex_normals()
aabb = mesh.get_axis_aligned_bounding_box()


# Get the minimum and maximum coordinates of the AABB
min_coords = aabb.get_min_bound()
max_coords = aabb.get_max_bound()

# Define the eight corners of the AABB

vertices = [
    [0, -.5, -.5],
    [0, -.5, -.5],
    [0, .5, -.5],
    [0, .5, -.5],
    [0, -.5, .5], 
    [0, -.5, .5],
    [0, .5, .5],
    [0, .5, .5]
]

print(vertices)
# Define the edges of the AABB
lines = [
    [0, 1],
    [1, 2],
    [2, 3],
    [3, 0],
    [4, 5],
    [5, 6],
    [6, 7],
    [7, 4],
    [0, 4],
    [1, 5],
    [2, 6],
    [3, 7]
]

# Create a LineSet to visualize the AABB
aabb_line_set = o3d.geometry.LineSet()
aabb_line_set.points = o3d.utility.Vector3dVector(vertices)
aabb_line_set.lines = o3d.utility.Vector2iVector(lines)

# Customize the bounding box appearance
line_set_color = np.array([1.0, 0.0, 0.0])  # Red color (R, G, B)
line_set_line_width = 5.0  # Adjust line width as needed

# Set the line set's color and line width
aabb_line_set.paint_uniform_color(line_set_color)
# aabb_line_set.line_width = line_set_line_width

# Visualize the object and its AABB
o3d.visualization.draw_geometries([mesh, aabb_line_set])

[[0, -0.5, -0.5], [0, -0.5, -0.5], [0, 0.5, -0.5], [0, 0.5, -0.5], [0, -0.5, 0.5], [0, -0.5, 0.5], [0, 0.5, 0.5], [0, 0.5, 0.5]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 


In [12]:
def get_surface(verticies, triangles):

    S = 0

    for t in triangles:

        a, b, c = verticies[t]

        S += get_triangle_surface(a - b, a - c)

    return S

 

def get_triangle_surface(v1, v2):

    Xab, Yab, Zab = v1

    Xac, Yac, Zac = v2

    return (((Yab * Zac - Zab * Yac)**2 + (Zab * Xac - Xab * Zac)**2 + (Xab * Yac - Yab * Xac)**2)**0.5) / 2

In [13]:
get_surface(np.asarray(mesh.vertices), np.asarray(mesh.triangles))

2.0950639904215542